# 如何使用内置工具和工具包

:::info 前提条件

本指南假定您熟悉以下概念：

- [LangChain工具](/docs/concepts/tools)
- [LangChain工具包](/docs/concepts/tools)

:::

## 工具

LangChain拥有大量第三方工具集合。请访问[工具集成](/docs/integrations/tools/)查看可用工具列表。

:::important

使用第三方工具时，请确保您了解该工具的工作原理和权限。
阅读其文档并检查从安全角度来看是否需要您做些什么。
更多信息请参阅我们的[安全](https://python.langchain.com/docs/security/)
指南。

:::

让我们试用[Wikipedia集成](/docs/integrations/tools/wikipedia/)。

In [1]:
!pip install -qU langchain-community wikipedia

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print(tool.invoke({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications 


该工具具有以下默认关联值：

In [ ]:
print(f"名称: {tool.name}")
print(f"描述: {tool.description}")
print(f"参数模式: {tool.args}")
print(f"直接返回结果?: {tool.return_direct}")

Name: wikipedia
Description: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
args schema: {'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}
returns directly?: False


## 自定义默认工具
我们还可以修改内置的名称、描述和参数的JSON模式。

在定义参数的JSON模式时，重要的是保持输入与函数相同，所以您不应该改变这一点。但您可以轻松地为每个输入定义自定义描述。

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from pydantic import BaseModel, Field


class WikiInputs(BaseModel):
    """Wikipedia工具的输入。"""

    query: str = Field(
        description="在Wikipedia中查询的问题，应该是3个或更少的单词"
    )


tool = WikipediaQueryRun(
    name="wiki-tool",
    description="在维基百科中查找信息",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)

print(tool.run("langchain"))

Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications 


In [ ]:
print(f"名称: {tool.name}")
print(f"描述: {tool.description}")
print(f"参数模式: {tool.args}")
print(f"直接返回结果?: {tool.return_direct}")

Name: wiki-tool
Description: look up things in wikipedia
args schema: {'query': {'description': 'query to look up in Wikipedia, should be 3 or less words', 'title': 'Query', 'type': 'string'}}
returns directly?: True


## 如何使用内置工具包

工具包是为特定任务设计的工具集合。它们提供了方便的加载方法。

所有工具包都公开了一个`get_tools`方法，该方法返回工具列表。

您通常应该这样使用它们：

```python
# 初始化工具包
toolkit = ExampleTookit(...)

# 获取工具列表
tools = toolkit.get_tools()
```